In [52]:
# -*- coding: utf-8 -*-

import pandas as pd
import numpy as np
from sklearn.feature_extraction.text import TfidfVectorizer

from sklearn.pipeline import Pipeline
from sklearn.grid_search import GridSearchCV 
from sklearn.grid_search import RandomizedSearchCV 

from sklearn.cross_validation import cross_val_score
from sklearn.cross_validation import StratifiedKFold

from sklearn.linear_model import LogisticRegression
from nltk.corpus import stopwords
from nltk.stem.snowball import RussianStemmer
import pymorphy2

import sys

reload(sys)
sys.setdefaultencoding('utf8')

In [61]:
morph = pymorphy2.MorphAnalyzer()
morph.parse(u'камерой,')

[Parse(word=u'\u043a\u0430\u043c\u0435\u0440\u043e\u0439,', tag=OpencorporaTag('UNKN'), normal_form=u'\u043a\u0430\u043c\u0435\u0440\u043e\u0439,', score=1.0, methods_stack=((<UnknAnalyzer>, u'\u043a\u0430\u043c\u0435\u0440\u043e\u0439,'),))]

In [59]:
stemmer = RussianStemmer()
stemmer.stem(u'камерой,')

u'\u043a\u0430\u043c\u0435\u0440\u043e\u0439,'

In [6]:
data = pd.read_csv('./data/data.csv')
del data['Unnamed: 0']

In [7]:
data.head()

,grade,link,text,content
0,5,https://technopoint.ru/product/42db0e393aab333...,"большой экран, мощный аккумулятор, режим для ч...",https://technopoint.ru/product/42db0e393aab333...
1,5,https://technopoint.ru/product/da39313444f8333...,Телефоном пользуюсь немного и это скорее перв...,https://technopoint.ru/product/da39313444f8333...
2,4,https://technopoint.ru/product/44b2f37e4035333...,"Цена, брал за 7900 руб. Яркий экран, хорошо ...",https://technopoint.ru/product/44b2f37e4035333...
3,5,https://technopoint.ru/product/305c1b434efe333...,Цельный качественный корпус с большой батарее...,https://technopoint.ru/product/305c1b434efe333...
4,5,https://technopoint.ru/product/0ecb5f413466333...,"камера, металл корпус , андроид отличный бюдже...",https://technopoint.ru/product/0ecb5f413466333...


In [8]:
data.grade.value_counts()

5    8766
4    2408
3    1060
1     652
2     640
Name: grade, dtype: int64

In [9]:
data['label'] = data.grade.apply(lambda x: 0 if x < 4 else 1)

In [10]:
data.label.value_counts()

1    11174
0     2352
Name: label, dtype: int64

In [11]:
corpus = data[['text', 'label']]

In [12]:
corpus.head()

,text,label
0,"большой экран, мощный аккумулятор, режим для ч...",1
1,Телефоном пользуюсь немного и это скорее перв...,1
2,"Цена, брал за 7900 руб. Яркий экран, хорошо ...",1
3,Цельный качественный корпус с большой батарее...,1
4,"камера, металл корпус , андроид отличный бюдже...",1


In [13]:
drop_indices = np.random.choice(corpus[corpus.label == 1].index, 8500, replace=False)
corpus = corpus.drop(drop_indices)
corpus.label.value_counts()

1    2674
0    2352
Name: label, dtype: int64

In [30]:
cv = StratifiedKFold(corpus.label, n_folds=10, shuffle=True, random_state=1)

pipeline = Pipeline([
    ('vectorizer', TfidfVectorizer()),
    ('classifier', LogisticRegression())
])

pipeline_params = {'classifier__penalty': ['l1', 'l2']}

grid = GridSearchCV(pipeline, pipeline_params, cv=cv, refit=True, verbose=1, n_jobs=4)

grid.fit(corpus.text.values.astype('U'), corpus.label)
best = grid.best_estimator_
print(
    "Accuracy (TfidfVectorizer + LogisticRegression): {}, params {}" . format(grid.best_score_, grid.best_params_)
)
grid.best_score_

0.980777761348514

In [31]:
grid.best_params_

{'classifier__penalty': 'l2'}

In [28]:
vectorizer = TfidfVectorizer(
    ngram_range=(1, 2), max_df=1.0, analyzer='word', use_idf=True,
    norm='l2', stop_words=stopwords.words('russian')
)
X_vect = vectorizer.fit_transform(corpus.text.values.astype('U'))

classifier = LogisticRegression(penalty='l2')
fitted = classifier.fit(X_vect, corpus.label)

In [29]:
X_test = vectorizer.transform([unicode('Пользуюсь 3 месяца.  На выбор повлиял большой 5,5 дюймовый  HD экран и мощный аккумулятор 4500 м Ач.  Экран действительно весьма большой и с качественной картинкой.  Удобно читать книги на таком экране.  Тем более что есть режим для комфортного чтения, свет на экране становится более теплым для глаз.  Можно к телефону подключать клавиатуру, компьютер и флэшку через  USB кабель.  Это мне часто помогает в работе.  Дома компа временно нет- отдал в ремонт,  а отчеты нужно отправлять вовремя, поэтому приходится пользоваться телефоном дома.  Аккумулятора хватает на 2 дня, заряжаю через день.  Камера снимает в разных режимах.  Я снимаю обычно серийную съемку, камера делает сразу 30 снимков, из них потом легче выбирать.')])
fitted.predict_proba(X_test)

array([[ 0.23646965,  0.76353035]])

In [30]:
fitted.predict(X_test)

array([1])